In [1]:
import torch
from torch_geometric.datasets import CoraFull
from torch_geometric.data import DataLoader
import aggregators

In [2]:
dataset = CoraFull(root="cora")
data = dataset[0]
loader = DataLoader(dataset, batch_size=32, shuffle=True)

Processing...
Done!
/home/paul/Documents/dev/ml-implementations/venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [4]:
class MeanAggregator(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(MeanAggregator, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.w = torch.nn.Parameter(torch.empty(2 * in_features, out_features))
        torch.nn.init.xavier_uniform_(self.w)

    def forward(self, input):
        x, sampled_neighbors = input
        neighbors_message = torch.mean(sampled_neighbors, dim=0)
        message = torch.concat((x, neighbors_message), 0)
        weighted_message = torch.matmul(message, self.w) 
        return torch.nn.functional.relu(weighted_message)


aggregators.py	cora  GraphSAGE.ipynb  main.py	__pycache__


In [ ]:
layer = aggregators.MeanAggregator(len(data.x[0]), 4)
node = data.x[0]
node_neighbors = torch.stack([data.x[v] for u, v in zip(data.edge_index[0], data.edge_index[1]) if data.edge_index[0][u] == 0])
print(layer((node, node_neighbors)))